In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from SALib.analyze import sobol
import warnings
from ema_workbench.analysis import feature_scoring
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    perform_experiments,
    Samplers,
    SequentialEvaluator,
)
# from dike_model_function import DikeNetwork 
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.analysis import prim
from ema_workbench.em_framework.parameters import Constant

warnings.filterwarnings("ignore")

In [1]:
dike_model, planning_steps = get_model_for_problem_formulation(6)

# Select the policy levers from the results
policy_levers = results[['SELECT LEVERS COLUMNS']] # Results haal je uit optimization
# Create a dict with the number of the candidate strategy as key and as value a dict containing the policy levers of that strategy
policy = {index: {p: row[p] for p in dike_model.levers} for index, row in policy_levers.iterrows()}


# ROELOFS CODE:
# zero_policy = {"DaysToThreat": 0}
# zero_policy.update({f"DikeIncrease {n}": 0 for n in planning_steps})
# zero_policy.update({f"RfR {n}": 0 for n in planning_steps})
# 
# pol0 = {}
# #
# for key in dike_model_1.levers:
#     s1, s2 = key.name.split("_")
#     pol0.update({key.name: zero_policy[s2]})
# 
# policy0 = Policy("Policy 0", **pol0)

IndentationError: expected an indented block after 'for' statement on line 8 (3133590997.py, line 12)

In [ ]:
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=15000,policies=list(policy.values()),uncertainty_sampling=Samplers.LHS)

experiments, outcomes = results

In [ ]:
df_outcomes = pd.DataFrame(outcomes)

In [ ]:
x = experiments

y = ((df_outcomes["A.5_Expected Number of Deaths"] > list(np.sort(df_outcomes["A.5_Expected Number of Deaths"]))[np.int64(0.80* len(df_outcomes["A.5_Expected Number of Deaths"]))])
    |(df_outcomes['A.5_Expected Annual Damage'] > list(np.sort(df_outcomes['A.5_Expected Annual Damage']))[np.int64(0.80* len(df_outcomes['A.5_Expected Annual Damage']))])
     )

prim_alg = prim.Prim(x, y, threshold=0.8,peel_alpha=0.1)
box1 = prim_alg.find_box()
box1.show_tradeoff(annotated=True)
plt.show()

In [ ]:
pd.DataFrame(box1.inspect(16, style='data')[0][1]).to_csv('data\A5_space_Prim3.csv')

In [ ]:
y = (
    (df_outcomes['Expected Number of Deaths'] > list(np.sort(df_outcomes["Expected Number of Deaths"]))[np.int64(0.8* len(df_outcomes['Expected Number of Deaths']))])
     |(df_outcomes['Expected Annual Damage'] > list(np.sort(df_outcomes["Expected Annual Damage"]))[np.int64(0.8* len(df_outcomes['Expected Annual Damage']))])
     )
# (df_feature_importance_out["Total Costs"] >= list(np.sort(df_feature_importance_out["Total Costs"]))[int(0.80* len(df_feature_importance_out["Total Costs"]))]) 
#      |
prim_alg = prim.Prim(x, y, threshold=0.95,peel_alpha=0.1)
box2 = prim_alg.find_box()
box2.show_tradeoff(annotated=True)
plt.show()

In [ ]:
pd.DataFrame(box2.inspect(16, style='data')[0][1]).to_csv('data\All_space_Prim3.csv')

In [ ]:
from Convience_fun import create_scenario
import multirange as mr

df_param_space_A5 = (pd.read_csv('data/A5_space_Prim3.csv').drop(0).rename(columns={"Unnamed: 0": "Variable"}))
df_param_space_all = (pd.read_csv('data/All_space_Prim3.csv').drop(0).rename(columns={"Unnamed: 0": "Variable"}))

# pd.read_csv('data/para_space_all.csv').drop(0).rename(columns={"Unnamed: 0": "Variable"})

dike_model, planning_steps = get_model_for_problem_formulation(6)


def create_scenario_from_box(all, a5):
    para_list = []
    for uncertainty in dike_model.uncertainties:
        # print((variable in df_param_space_A5.Variable))
        variable = uncertainty.name
        # print(variable)
        if (variable in list(df_param_space_A5.Variable)) & (variable in list(df_param_space_all.Variable)) & (
                all + a5 == 2):
            min_1 = float(
                df_param_space_A5[df_param_space_A5['Variable'] == variable][df_param_space_A5.columns[1]].values[
                    0]) * 10000
            max_1 = float(
                df_param_space_A5[df_param_space_A5['Variable'] == variable][df_param_space_A5.columns[2]].values[
                    0]) * 10000

            min_2 = float(
                df_param_space_all[df_param_space_all['Variable'] == variable][df_param_space_all.columns[1]].values[
                    0]) * 10000
            max_2 = float(
                df_param_space_all[df_param_space_all['Variable'] == variable][df_param_space_all.columns[2]].values[
                    0]) * 10000

            boundaries = mr.overlap(range(int(min_1), int(max_1)), range(int(min_2), int(max_2)))
            para_list.append((boundaries.start + (boundaries.stop - boundaries.start) / 2) / 10000)
            print((boundaries.start + (boundaries.stop - boundaries.start) / 2) / 10000)
        elif (variable in list(df_param_space_A5.Variable)) & (a5 == True):
            min_1 = float(
                df_param_space_A5[df_param_space_A5['Variable'] == variable][df_param_space_A5.columns[1]].values[0])
            max_1 = float(
                df_param_space_A5[df_param_space_A5['Variable'] == variable][df_param_space_A5.columns[2]].values[0])

            para_list.append((min_1 + (max_1 - min_1) / 2))
        elif (variable in list(df_param_space_all.Variable)) & (all == True):
            min_2 = float(
                df_param_space_all[df_param_space_all['Variable'] == variable][df_param_space_all.columns[1]].values[0])
            max_2 = float(
                df_param_space_all[df_param_space_all['Variable'] == variable][df_param_space_all.columns[2]].values[0])

            para_list.append((min_2 + (max_2 - min_2) / 2))
        elif variable.startswith("d"):
            para_list.append(3.5)
        elif variable == "A.0_ID flood wave shape":
            para_list.append(17)
        elif variable.endswith("Brate"):
            para_list.append(1.5)
        else:
            para_list.append((uncertainty.lower_bound + (uncertainty.upper_bound - uncertainty.lower_bound) / 2))

    return create_scenario(para_list)

Scenarios_MOEA = [create_scenario_from_box(all=True, a5=True), create_scenario_from_box(all=True, a5=False),
                  create_scenario_from_box(all=False, a5=True)]
import pickle

with open("data\ScenariosOpenExplo", 'wb') as file:
    pickle.dump(Scenarios_MOEA, file) 